# Setup

In [3]:
%ls

 Volume in drive D is New Volume
 Volume Serial Number is 6085-9B4A

 Directory of d:\hcmus\introduction-to-algorithm-complexity-and-analysis\binsketch-algorithm

09-Jan-26  12:41 AM    <DIR>          .
08-Jan-26  08:37 PM    <DIR>          ..
08-Jan-26  11:56 PM                75 .gitignore
09-Jan-26  12:43 AM    <DIR>          .venv
09-Jan-26  12:41 AM                 0 bin_sketch.ipynb
09-Jan-26  12:07 AM             7,903 convert.py
09-Jan-26  12:08 AM    <DIR>          data
08-Jan-26  11:57 PM             1,673 download_dataset.py
08-Jan-26  03:40 PM                 0 main.py
08-Jan-26  10:22 PM    <DIR>          raw
09-Jan-26  12:35 AM                25 README.md
09-Jan-26  12:42 AM                51 requirements.txt
09-Jan-26  12:44 AM    <DIR>          src
               7 File(s)          9,727 bytes
               6 Dir(s)  472,715,218,944 bytes free


# Prepare dataset

In [4]:
!python download_dataset.py

Processing: https://drive.google.com/drive/folders/1ARBY9cIGj_jigi5Y88CtUy-GMj2clrXj
Retrieving folder 10Y_7o78v8HhztcE7bzgV3M94N0vXjt_U bbc
Processing file 1X4DnqhkCKLHd3qkup4cG-0NxjZ_If_wt docword.bbc.txt.gz
Processing file 1kO9AOWWuACtNsgmA9yyp9C6pSoB8KW3U vocab.bbc.txt
Retrieving folder 1D9mMF6ealOinLAsmwXsZs9IMNPBqzuM0 enron
Processing file 1JuUxpaQRAl1yZGqb3xcSP8nfK3_1NZ8y docword.enron.txt.gz
Processing file 16Rn70xrTnYOIkVm2mz4ICR2bEbyzyvGQ vocab.enron.txt
Retrieving folder 1YMxNXk2-7Ok1_3gnIguPwWt365C5X7Et kos
Processing file 1c1bJ-eX5Rp729zGSeGGXqwyor4DfWsrx docword.kos.txt.gz
Processing file 1YL0wnFKLJz-h6emVWHYDfBET5cAZhdKi vocab.kos.txt
Retrieving folder 17JU-ouMBLAUilZiaxE1xzeKfpFvDy9PA nytimes
Processing file 1gsmnfyNEAtA_3kdU5GMhwnUX-vOlc_9A docword.nytimes.txt.gz
Processing file 1jAnAFekn8u-e_FO1tsElhr-gP_dvSXkx vocab.nytimes.txt

[SUCCESS] Folder contents downloaded to: ./raw


Retrieving folder contents
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1X4DnqhkCKLHd3qkup4cG-0NxjZ_If_wt
To: d:\hcmus\introduction-to-algorithm-complexity-and-analysis\binsketch-algorithm\raw\bbc\docword.bbc.txt.gz

  0%|          | 0.00/490k [00:00<?, ?B/s]
100%|██████████| 490k/490k [00:00<00:00, 2.07MB/s]
100%|██████████| 490k/490k [00:00<00:00, 2.06MB/s]
Downloading...
From: https://drive.google.com/uc?id=1kO9AOWWuACtNsgmA9yyp9C6pSoB8KW3U
To: d:\hcmus\introduction-to-algorithm-complexity-and-analysis\binsketch-algorithm\raw\bbc\vocab.bbc.txt

  0%|          | 0.00/77.2k [00:00<?, ?B/s]
100%|██████████| 77.2k/77.2k [00:00<00:00, 425kB/s]
100%|██████████| 77.2k/77.2k [00:00<00:00, 425kB/s]
Downloading...
From: https://drive.google.com/uc?id=1JuUxpaQRAl1yZGqb3xcSP8nfK3_1NZ8y
To: d:\hcmus\introduction-to-algorithm-complexity-and-analysis\binsketch-algorithm\raw\enron\docword

In [5]:
!python convert.py


Processing BBC dataset
Loading vocabulary from ./raw/bbc/vocab.bbc.txt...
Loading document-word data from ./raw/bbc/docword.bbc.txt.gz...
Error processing bbc: invalid literal for int() with base 10: './\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

Processing ENRON dataset
Loading vocabulary from ./raw/enron/vocab.enron.txt...
Loading document-word data from ./raw/enron/docword.enron.txt.gz...
  Documents: 39861, Words: 28102, Non-zeros: 3710420
  Sampling 5000 documents out of 39861
Creating sparse matrix...
Converting to DataFrame in batches of 1000 documents...
Concatenating batches...
DataFrame shape: (5000, 28102)
Sparsity: 99.74%

ENRON DataFrame Preview:
             aaa  aaas  aactive  aadvantage  aaker  aap  aapg  aaron  aarp  aas
document_id                                                                    
0              0


Reading sparse data: 100%|██████████| 3710420/3710420 [00:02<00:00, 1551664.90 entries/s]

Processing batches: 100%|██████████| 5/5 [00:00<00:00, 71.65 batch/s]

Reading sparse data: 100%|██████████| 353160/353160 [00:00<00:00, 607531.58 entries/s]

Processing batches: 100%|██████████| 7/7 [00:00<00:00, 164.46 batch/s]

Reading sparse data: 100%|██████████| 69679427/69679427 [00:35<00:00, 1943511.39 entries/s]

Processing batches: 100%|██████████| 1/1 [00:00<00:00,  4.42 batch/s]
